<a href="https://colab.research.google.com/github/ralsouza/apache_spark_real_time_analytics/blob/master/notebooks/05_Spark_grouping_by_pair_rdd_broadcast_accumulators_and_partitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.

In [18]:
# Install the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [19]:
# Environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
# Libraries and Context Setup
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)

# Instance Spark Session
spark = SparkSession.builder.master('local').appName('My-SparkSQL').getOrCreate()

# Create the SQL Context
sqlContext = pyspark.SQLContext(sc)

In [5]:
# Check context
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


# Grouping by pair RDD

In [25]:
# Import cars's csv
cars = sc.textFile('/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/carros.csv')

In [26]:
cars.take(20)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348',
 'mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,37,41,5389',
 'honda,gas,std,two,hatchback,fwd,four,60,5500,38,42,5399',
 'nissan,gas,std,two,sedan,fwd,four,69,5200,31,37,5499',
 'dodge,gas,std,two,hatchback,fwd,four,68,5500,37,41,5572',
 'plymouth,gas,std,two,hatchback,fwd,four,68,5500,37,41,5572',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,31,38,6095',
 'mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,31,38,6189',
 'dodge,gas,std,four,hatchback,fwd,four,68,5500,31,38,6229',
 'plymouth,gas,std,four,hatchback,fwd,four,68,5500,31,38,6229',
 'chevrolet,gas,std,two,hatchback,fwd,four,70,5400,38,43,6295',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,31,3

In [27]:
# Creating a pair rdd with columns 0 and 7
cars_pair_rdd = cars.map(lambda x: (x.split(',')[0],x.split(',')[7]))

In [28]:
# Check result
cars_pair_rdd.take(20)

[('MAKE', 'HP'),
 ('subaru', '69'),
 ('chevrolet', '48'),
 ('mazda', '68'),
 ('toyota', '62'),
 ('mitsubishi', '68'),
 ('honda', '60'),
 ('nissan', '69'),
 ('dodge', '68'),
 ('plymouth', '68'),
 ('mazda', '68'),
 ('mitsubishi', '68'),
 ('dodge', '68'),
 ('plymouth', '68'),
 ('chevrolet', '70'),
 ('toyota', '62'),
 ('dodge', '68'),
 ('honda', '58'),
 ('toyota', '62'),
 ('honda', '76')]

In [29]:
# Select the header
header = cars_pair_rdd.first()

# Select all data except the header
cars_pair_rdd2 = cars_pair_rdd.filter(lambda line: line != header)

In [30]:
# Search the HP value by car's brand and add 1 to each "Make/HP" row
cars_pair_rdd3 = cars_pair_rdd2.mapValues(lambda x: (x,1))
cars_pair_rdd3.collect()

[('subaru', ('69', 1)),
 ('chevrolet', ('48', 1)),
 ('mazda', ('68', 1)),
 ('toyota', ('62', 1)),
 ('mitsubishi', ('68', 1)),
 ('honda', ('60', 1)),
 ('nissan', ('69', 1)),
 ('dodge', ('68', 1)),
 ('plymouth', ('68', 1)),
 ('mazda', ('68', 1)),
 ('mitsubishi', ('68', 1)),
 ('dodge', ('68', 1)),
 ('plymouth', ('68', 1)),
 ('chevrolet', ('70', 1)),
 ('toyota', ('62', 1)),
 ('dodge', ('68', 1)),
 ('honda', ('58', 1)),
 ('toyota', ('62', 1)),
 ('honda', ('76', 1)),
 ('chevrolet', ('70', 1)),
 ('nissan', ('69', 1)),
 ('mitsubishi', ('68', 1)),
 ('dodge', ('68', 1)),
 ('plymouth', ('68', 1)),
 ('mazda', ('68', 1)),
 ('isuzu', ('78', 1)),
 ('mazda', ('68', 1)),
 ('nissan', ('69', 1)),
 ('honda', ('76', 1)),
 ('toyota', ('62', 1)),
 ('toyota', ('70', 1)),
 ('mitsubishi', ('88', 1)),
 ('subaru', ('73', 1)),
 ('nissan', ('55', 1)),
 ('subaru', ('82', 1)),
 ('honda', ('76', 1)),
 ('toyota', ('70', 1)),
 ('honda', ('76', 1)),
 ('honda', ('76', 1)),
 ('nissan', ('69', 1)),
 ('nissan', ('69', 1)),
 

In [31]:
# Apply the reduction by key (reduceByKey)
# Calculates the HP total by maker and the total of vehicles by maker
makers = cars_pair_rdd3.reduceByKey(lambda x,y: (int(x[0]) + int(y[0]), x[1] + y[1]))
makers.collect()

[('chevrolet', (188, 3)),
 ('mazda', (1390, 16)),
 ('mitsubishi', (1353, 13)),
 ('nissan', (1846, 18)),
 ('dodge', (675, 8)),
 ('plymouth', (607, 7)),
 ('saab', (760, 6)),
 ('volvo', (1408, 11)),
 ('alfa-romero', (376, 3)),
 ('mercedes-benz', (1170, 8)),
 ('jaguar', (614, 3)),
 ('subaru', (1035, 12)),
 ('toyota', (2969, 32)),
 ('honda', (1043, 13)),
 ('isuzu', (168, 2)),
 ('volkswagen', (973, 12)),
 ('peugot', (1098, 11)),
 ('audi', (687, 6)),
 ('bmw', (1111, 8)),
 ('mercury', ('175', 1)),
 ('porsche', (764, 4))]

# Accumulators and Broadcast
The Spark make a copy of the code to process the data and execute these copies, one by node in the cluster. **Any variable maked in the the code is local in the node**. The Spark generates copies these local variables, one in each node, that performs independently. But if we need that same variable to be manipulated only way across the cluster? We will use the broadcast and accumulators.

These variables will be visible to whole spark cluster.
*   **Broarcast variable:** read-only, is shared in whole cluster
*   **Accumulator variable:** is shared in whole cluster, but can be refreshed in each node of cluster




In [ ]:
# Calc the HP mean dividing by total count
makers.mapValues(lambda x: int(x[0])/int(x[1])).collect()

[('chevrolet', 62.666666666666664),
 ('mazda', 86.875),
 ('mitsubishi', 104.07692307692308),
 ('nissan', 102.55555555555556),
 ('dodge', 84.375),
 ('plymouth', 86.71428571428571),
 ('saab', 126.66666666666667),
 ('volvo', 128.0),
 ('alfa-romero', 125.33333333333333),
 ('mercedes-benz', 146.25),
 ('jaguar', 204.66666666666666),
 ('subaru', 86.25),
 ('toyota', 92.78125),
 ('honda', 80.23076923076923),
 ('isuzu', 84.0),
 ('volkswagen', 81.08333333333333),
 ('peugot', 99.81818181818181),
 ('audi', 114.5),
 ('bmw', 138.875),
 ('mercury', 175.0),
 ('porsche', 191.0)]

In [ ]:
# Import cars's csv
cars = sc.textFile('/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/carros.csv')

In [ ]:
# Initializing the Accumulator variable
sedan_count = sc.accumulator(0)
hatchback_count = sc.accumulator(0)

In [ ]:
# Initializing the Bradcast variable - read-only
sedan_text = sc.broadcast('sedan')
hatchback_text = sc.broadcast('hatchback')

In [ ]:
sedan_text.value

'sedan'

In [ ]:
def split_lines(line):

  # To use the Accumulators variables
  global sedan_count
  global hatchback_count

  # Use the broadcast variable to compare and add a count to accumulator
  if sedan_text.value in line:
    sedan_count += 1
  if hatchback_text.value in line:
    hatchback_count += 1

  return line.split(',')

In [ ]:
# Map 
split_data = cars.map(split_lines)

In [ ]:
# Action to execute the transformation (lazy evaluation)
split_data.count()
print(sedan_count,hatchback_count)

92 67


In [ ]:
split_data.count()

198

In [ ]:
print(sedan_count,hatchback_count)

184 134


# Partitions

In [32]:
makers.getNumPartitions()

2

In [33]:
#  Defining the partition number
coll_data = sc.parallelize([3,5,4,7,4], 3)
coll_data.cache()
coll_data.count()

5

In [34]:
# Check the partitions
coll_data.getNumPartitions()

3

In [35]:
# Check de default config to partitions
print(sc.defaultParallelism)

2
